In [1]:
import json
import importlib
from transformers import T5Tokenizer
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer, Regex
from tokenizers.normalizers import NFKC, Sequence
from tokenizers.pre_tokenizers import Metaspace, Split, WhitespaceSplit

import re
import src.paper_to_equation.Generation.Equation_BaseDataset 


c:\Users\kyanj\anaconda3\envs\fyp_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
importlib.reload(src.paper_to_equation.Generation.Equation_BaseDataset)
from src.paper_to_equation.Generation.Equation_BaseDataset import BaseDataset

Confirming the default tokenizer is unsuitable

In [4]:
tokenizer = T5Tokenizer.from_pretrained("t5-base")
test_text = """
def my_function(x, y): return x + y
class MyClass:
    def method(self): pass
import numpy as np
<math><msup><mi>x</mi><mn>2</mn></msup></math>
"""
mml = """
<mml:mi>h</mml:mi>
<mml:mo>=</mml:mo>
<mml:mrow>
<mml:msub>
<mml:mi>h</mml:mi>
<mml:mi>c</mml:mi>
</mml:msub>
<mml:mo>+</mml:mo>
<mml:msub>
<mml:mi>h</mml:mi>
<mml:mi>g</mml:mi>
</mml:msub>
</mml:mrow>
  """

py = """
h = Symbol('h')
h_g = Symbol('h_g')
h_c = Symbol('h_c')
e = Eq(h, h_g + h_c)"""

tokens = tokenizer.tokenize(mml)
print(tokenizer.tokenize(test_text))
print(len(tokenizer.tokenize(test_text))) 


['▁de', 'f', '▁my', '_', 'function', '(', 'x', ',', '▁', 'y', ')', ':', '▁return', '▁', 'x', '▁+', '▁', 'y', '▁class', '▁My', 'Class', ':', '▁de', 'f', '▁method', '(', 'self', ')', ':', '▁pass', '▁import', '▁nu', 'mp', 'y', '▁as', '▁', 'n', 'p', '▁', '<', 'mat', 'h', '>', '<', 'm', 's', 'up', '>', '<', 'm', 'i', '>', 'x', '<', '/', 'm', 'i', '>', '<', 'm', 'n', '>', '2', '<', '/', 'm', 'n', '>', '<', '/', 'm', 's', 'up', '>', '<', '/', 'mat', 'h', '>']
79


Dataset class

In [9]:
class TokenizerDataset(BaseDataset):
    def __init__(self, num, filepath):
        super().__init__(num, filepath)

    def get_columns(self):
        return ["mathml", "python"]

    def map_atomic_tokens(self, dataset):

        tag_map = {"<mml:mo>": "<MO>", "</mml:mo>": "</MO>",
                   "<mml:mi>": "<MI>", "</mml:mi>": "</MI>",
                   "<mml:msub>": "<MSUB>", "</mml:msub>": "</MSUB>",
                   "<mml:msup>": "<MSUP>", "</mml:msup>": "</MSUP>",
                   "<mml:mrow>": "<MROW>", "</mml:mrow>": "</MROW>", 
                   "<mml:mfrac>": "<MFRAC>", "</mml:mfrac>": "</MFRAC>"}
        
        for entry in dataset:
            mathml = entry["mathml"]
            for tag, token in tag_map.items():
                mathml = mathml.replace(tag, token)
            entry["mathml"] = mathml
        
        return dataset
    
    def json_to_dataset(self):
        with open(self.filepath) as f:
            data = json.load(f)
        
        return data
    
    def get_tag_list():
        return ["<mml:mo>", "</mml:mo>",
                "<mml:mi>", "</mml:mi>",
                "<mml:msub>", "</mml:msub>",
                "<mml:msup>", "</mml:msup>",
                "<mml:mrow>", "</mml:mrow>",
                "<mml:mfrac>", "</mml:mfrac>"
                "<mml:mtext>", "</mml:mtext>"]
    
    
    def extract_tags(self, data):
        tags = set()
        for entry in data:
            mathml = entry["mathml"]
            for tag in mathml:
                if tag not in tags:
                    tags.append(tag)
        return tags
        
    def data_iterator(self, data, batch_size):
        for i in range(0, len(data), batch_size):
            yield [sample["mathml"] for sample in data[i:i+batch_size]]  

In [12]:
td = TokenizerDataset(10, "Data/tokenizer_dataset.json")
# td.create()
data = td.json_to_dataset()
# mapped_data = td.map_atomic_tokens(data)
for entry in td.data_iterator(data, 10):
    print(entry)




['<mml:mi>h</mml:mi>\n<mml:mo>=</mml:mo>\n<mml:mrow>\n<mml:msup>\n<mml:mtext>exp</mml:mtext>\n<mml:mrow>\n<mml:mrow>\n<mml:mo>-</mml:mo>\n<mml:mi>Ε</mml:mi>\n</mml:mrow>\n<mml:mo>+</mml:mo>\n<mml:mi>ν</mml:mi>\n</mml:mrow>\n</mml:msup>\n<mml:mo>-</mml:mo>\n<mml:mrow>\n<mml:mi>sin</mml:mi>\n<mml:mfenced>\n<mml:msub>\n<mml:mi>f</mml:mi>\n<mml:mrow>\n<mml:mi>t</mml:mi>\n<mml:mi>S</mml:mi>\n</mml:mrow>\n</mml:msub>\n</mml:mfenced>\n</mml:mrow>\n<mml:mo>+</mml:mo>\n<mml:mrow>\n<mml:mi>tan</mml:mi>\n<mml:mfenced>\n<mml:msub>\n<mml:mi>ο</mml:mi>\n<mml:mrow>\n<mml:mi>T</mml:mi>\n<mml:mi>λ</mml:mi>\n</mml:mrow>\n</mml:msub>\n</mml:mfenced>\n</mml:mrow>\n</mml:mrow>', '<mml:mi>ν</mml:mi>\n<mml:mo>=</mml:mo>\n<mml:mrow>\n<mml:munderover>\n<mml:mo>∑</mml:mo>\n<mml:mrow>\n<mml:mi>κ</mml:mi>\n<mml:mo>=</mml:mo>\n<mml:mn>9</mml:mn>\n</mml:mrow>\n<mml:msub>\n<mml:mi>y</mml:mi>\n<mml:mi>Δ</mml:mi>\n</mml:msub>\n</mml:munderover>\n<mml:mfenced>\n<mml:mrow>\n<mml:msqrt>\n<mml:mrow>\n<mml:mi>Λ</mml:mi>\n<

In [22]:
tokenizer = Tokenizer(models.Unigram())
tokenizer.normalizer = normalizers.Sequence(
    [normalizers.Replace("``", '"'), normalizers.Replace("''", '"')] # don't lowercase
)
tokenizer.normalizer.normalize_str(py)

tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()
tokenizer.pre_tokenizer.pre_tokenize_str(py)



[('▁\nh', (0, 2)),
 ('▁=', (2, 4)),
 ("▁Symbol('h')\nh_g", (4, 20)),
 ('▁=', (20, 22)),
 ("▁Symbol('h_g')\nh_c", (22, 40)),
 ('▁=', (40, 42)),
 ("▁Symbol('h_c')\ne", (42, 58)),
 ('▁=', (58, 60)),
 ('▁Eq(h,', (60, 66)),
 ('▁h_g', (66, 70)),
 ('▁+', (70, 72)),
 ('▁h_c)', (72, 77))]

In [ ]:
class MathMLTokenizer(Tokenizer):
    def __init__(self, data):
        self.data = data
       
    def normalise(self):
        pass

In [3]:
tokenizer = Tokenizer(models.BPE())                  

class MathMLPyTokenizer(pre_tokenizers.PreTokenizer):
    def __init__(self):
        pass
    
    def pre_tokenize(self, data):
        
        patterns = [
            # HTML/XML tags
            r'<[^>]+>',
            # Python string literals
            r'"""[^"]*"""', r"'''[^']*'''", r'"[^"]*"', r"'[^']*'",
            # Python keywords and operators
            r'\bdef\b', r'\bclass\b', r'\bfor\b', r'\bwhile\b', r'\bif\b', r'\belif\b', r'\belse\b',
            r'\breturn\b', r'\bimport\b', r'\bfrom\b', r'\bas\b', r'\bwith\b', r'\btry\b', r'\bexcept\b',
            # Common Python syntax elements
            r'==', r'!=', r'<=', r'>=', r'\+=', r'-=', r'\*=', r'/=', 
            r'=>', r'->',  # Function type hints and lambdas
            r'\bSymbol\b', r'\bEq\b', r'\bexp\b', r'\bsin\b', r'\bcos\b', r'\btan\b', r'\bdiff\b',
            
            # Indentation (important for Python)
            r'^\s+'
        ]

        combined_pattern = '|'.join(f'({p})' for p in patterns)
        regex = re.compile(combined_pattern, re.MULTILINE)
        
        splits = []
        for text, offset in data:
            last_end = 0
            for match in regex.finditer(text):
                start, end = match.span()
                
                if start > last_end:
                    # Add text before the special token
                    splits.append((text[last_end:start], last_end + offset))
                
                # Add the special token as a whole
                splits.append((text[start:end], start + offset))
                last_end = end
            
            if last_end < len(text):
                # Add remaining text
                splits.append((text[last_end:], last_end + offset))
                
        return splits

x = MathMLPyTokenizer()

tokens = tokenizer.tokenize(mml)
print(tokens)


TypeError: No constructor defined

In [5]:
# Create a base tokenizer (BPE, WordPiece, or Unigram)
tokenizer = Tokenizer(models.BPE())

# Define patterns we want to protect during tokenization
text_patterns = [
            # HTML/XML tags
            r'<[^>]+>',
            # Python string literals
            r'"""[^"]*"""', r"'''[^']*'''", r'"[^"]*"', r"'[^']*'",
            # Python keywords and operators
            r'\bdef\b', r'\bclass\b', r'\bfor\b', r'\bwhile\b', r'\bif\b', r'\belif\b', r'\belse\b',
            r'\breturn\b', r'\bimport\b', r'\bfrom\b', r'\bas\b', r'\bwith\b', r'\btry\b', r'\bexcept\b',
            # Common Python syntax elements
            r'==', r'!=', r'<=', r'>=', r'\+=', r'-=', r'\*=', r'/=', 
            r'=>', r'->',  # Function type hints and lambdas
            r'\bSymbol\b', r'\bEq\b', r'\bexp\b', r'\bsin\b', r'\bcos\b', r'\btan\b', r'\bdiff\b', # MathML
            # Indentation (important for Python)
            r'^\s+',
            r'\s+' # Whitespace
        ]

# Set up the pre-tokenizer using Split with pattern
pattern = '|'.join(text_patterns)
split_pre_tokenizer = Split(pattern=Regex(pattern), behavior="isolated")
# split_pre_tokenizer = WhitespaceSplit()
tokenizer.pre_tokenizer = split_pre_tokenizer

# Set up other components 
tokenizer.normalizer = Sequence([NFKC()])
tokenizer.decoder = decoders.ByteLevel()

test_text = """
def my_function(x, y): return x + y
class MyClass:
    def method(self): pass
import numpy as np
<math><msup><mi>x</mi><mn>2</mn></msup></math>
"""

check = tokenizer.pre_tokenizer.pre_tokenize_str(test_text)
print(check, len(check))
output = tokenizer.encode(test_text)
print(output.tokens)
print(output.ids)


[('\n', (0, 1)), ('def', (1, 4)), (' ', (4, 5)), ('my_function(x,', (5, 19)), (' ', (19, 20)), ('y):', (20, 23)), (' ', (23, 24)), ('return', (24, 30)), (' ', (30, 31)), ('x', (31, 32)), (' ', (32, 33)), ('+', (33, 34)), (' ', (34, 35)), ('y', (35, 36)), ('\n', (36, 37)), ('class', (37, 42)), (' ', (42, 43)), ('MyClass:', (43, 51)), ('\n    ', (51, 56)), ('def', (56, 59)), (' ', (59, 60)), ('method(self):', (60, 73)), (' ', (73, 74)), ('pass', (74, 78)), ('\n', (78, 79)), ('import', (79, 85)), (' ', (85, 86)), ('numpy', (86, 91)), (' ', (91, 92)), ('as', (92, 94)), (' ', (94, 95)), ('np', (95, 97)), ('\n', (97, 98)), ('<math>', (98, 104)), ('<msup>', (104, 110)), ('<mi>', (110, 114)), ('x', (114, 115)), ('</mi>', (115, 120)), ('<mn>', (120, 124)), ('2', (124, 125)), ('</mn>', (125, 130)), ('</msup>', (130, 137)), ('</math>', (137, 144)), ('\n', (144, 145))] 44
[]
[]


In [14]:
test_text = """def my_function(x, y): return x + y
class MyClass:
    def method(self): pass
import numpy as np
<math><msup><mi>x</mi><mn>2</mn></msup></math>
"""

# Your combined regex pattern
tag_pattern = r'<[^>]+>'
python_patterns = [
    r'def\s+\w+\([^)]*\)', r'class\s+\w+', r'import\s+\w+', r'from\s+\w+\s+import',
    r'if\s+.*:', r'for\s+.*:', r'while\s+.*:', r'try:', r'except\s+.*:',
]
pattern = '|'.join([tag_pattern] + python_patterns)

# Test regex separately
regex = re.compile(pattern)
matches = regex.findall(test_text)
print("Matches:", matches)


Matches: ['def my_function(x, y)', 'class MyClass', 'def method(self)', 'import numpy', '<math>', '<msup>', '<mi>', '</mi>', '<mn>', '</mn>', '</msup>', '</math>']
